In [1]:
import torch.optim as optim
import torch, torchvision
import torch.nn as nn
from torch.autograd import Variable
from models import CNN

torch.set_default_tensor_type('torch.DoubleTensor')

from MLDataTools.image_normalization import trainloader

In [2]:
torch.cuda.device_count()
torch.cuda.current_device()

/home/user/anaconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:97: UserWarning: 
    Found GPU0 GeForce GTX 960M which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


0

In [3]:
DATA_DIR = '/home/user/HDev Dropbox/Projects/YNet_ready_data/yeast_v4'

In [4]:
model = CNN()


In [5]:
criterion = nn.CrossEntropyLoss()
optim = optim.Adam(model.parameters())
next(model.parameters()).is_cuda


False

In [16]:
loader = trainloader(DATA_DIR)

In [17]:
for epoch in range(5):  

    running_loss = 0.0
    for i,data in enumerate(loader, 0):
        
        # get the inputs
        inputs, labels = Variable(data[0]), Variable(data[1].type(torch.LongTensor))
        print("train loop: ",inputs.size())
        print("train loop: ",labels.size())

        labels = labels.squeeze_()
        print("train loop: ",labels.size())

        # zero the parameter gradients
        optim.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        print("train loop: ",outputs.size())

        loss = criterion(outputs, labels)
        loss.backward()
        optim.step()

        # print statistics
        if i % 5 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

length of yeast dataset: [714]
train loop:  torch.Size([140, 2, 200, 200])
train loop:  torch.Size([140])
train loop:  torch.Size([140])
torch.Size([140, 12, 47, 47])
train loop:  torch.Size([140, 4])
[1,     1] loss: 0.000
train loop:  torch.Size([140, 2, 200, 200])
train loop:  torch.Size([140])
train loop:  torch.Size([140])


RuntimeError: $ Torch: not enough memory: you tried to allocate 2GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/TH/THGeneral.c:253